# Author's Comment
The Comments below is wrtten by the kaggle user, who uploaded this dataset.
First thing I want to tell you is that the purpose of this notebook is to predict one's victory based on the conditions written on the datasets. And it'll be achieved by a classfication, and trendy XGboost model to get it better.

The second thing is that, I repeat these on my notebook just to let you know that I just followed the guidlines referred by the user.
If I continue to revise my model and work on it, it'd be much better to predict the victory of the battle.




## History of Battle
This dataset is a cleaned-up and modernized version of "CAA Database of Battles, Version 1990", shortnamed "CDB90". It contains information on over 600 battles that were fought between 1600 AD and 1973 AD. Descriptive data include battle name, date, and location; the strengths and losses on each side; identification of the victor; temporal duration of the battle; and selected environmental and tactical environment descriptors (such as type of fortifications, type of tactical scheme, weather conditions, width of front, etc.).


## Content
The data contained therein is split across several files. The most important of these is battles.csv, which is lists and gives information about the battles themselves. Files marked enum describe the keys used by specific fields. Other files provide additional context.

## Inspiration
-How often were battles fought in various weather conditions?

-How often did an attacker or defender achieve the element of surprise?

-Did it have a significant effect on the outcome?

-Did prepared fortifications have a significant effect on outcomes?


# Features

Before we get to the project, I'll make a referrence of the features to make it easier to understand the table.

-aeroa: air superitority

-cea: how much the actor favors attack or defense

-codead: total strength

-crit: ???

-post: how much defense prepared

-postype: posture taken

-pri: the form of tactics

-wina: 0-draw/1-attacker win/ -1 - attacker loss

-reso: result of the battle

-surpa: how much surprise achieved by attacker/defender

front_width: the features in it are unknown


### Terrain
-terra1 : None/Rugged/Rollig/Flat

-terra2 : Bare/Desert/Heavily Wooded/Mixed

-terra3 : None/ Marsh or Swamp/ Urban /Dunes

### Weather


-wx1: Dry/wet

-wx2: Heavy Precipitatiion/Sunny/Light Precipitation/Overcast

-wx3: hot/cold/temperate

-wx4: Spring/Summer/Fall/Winter

-wx5: Tropical/Desert/Temperate



# Data Overview

As mentioned above, the first phase will be just using 4 features.

-weather

-terrain

-the element of surprise

-fortification



Any other background knowledge will be used later on.
Before that, let us first overview the dataset.

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploring Dataset

## Tables

In [ ]:
battle = pd.read_csv('/kaggle/input/database-of-battles/battles.csv')
display(battle.tail())
print(battle.columns)

In [ ]:
terrain = pd.read_csv('/kaggle/input/database-of-battles/terrain.csv')
display(terrain.tail())
print(terrain.columns)

In [ ]:
weather = pd.read_csv('/kaggle/input/database-of-battles/weather.csv')
display(weather.tail())
print(weather.columns)

There are only 4 features that we're interested in, so we are gonna merge these tables and have only the features left.

-weather
-terrain
-fortification
-element of surprise achieved or not

In [ ]:
#The revised dataset that contains all information needed

df = pd.merge(battle, terrain , on="isqno")
df = pd.merge(df, weather, on = "isqno")
df.set_index('isqno', inplace = True)
df = df[['surpa', 'post1', 'post2', 'wx1', 'wx2', 'wx3', 'wx4', 'wx5', 'terra1', 'terra2', 'terra3', 'aeroa' , 'wina']]

display(df.tail())

In [ ]:
# Based on what I searched for, Nan datas in 'wina' are supposed to be -1.(attacker loss)
df['wina'] = df['wina'].fillna(-1)
display(df)

The reason the amount of the datas increased is that each war consists of a few battles, and each battle has its own condition.
But it seems that the datas sharing same isqno have the same 'wina' value, so we don't need to set up the function to fix it.

* Different condtions but same result! Need to think about it one day

## Missing Data


Now we'll check missing data and se the tables in the form easy to analyse.

### Element of Surprise achieved or not

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
surprise = df[['surpa', 'wina']]

msno.matrix(surprise)
plt.show()

print(surprise.isnull().sum())

### Terrain

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
terrain = df[['terra1', 'terra2', 'terra3', 'wina']]

msno.matrix(terrain)
plt.show()

print(terrain.isnull().sum()) 

### Weather

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
weather = df[['wx1', 'wx2', 'wx3', 'wx4', 'wx5','wina']]

msno.matrix(weather)
plt.show()

print(weather.isnull().sum())

### Formation of Defense

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.
fortification = df[['post1', 'post2' ,'wina']]
msno.matrix(fortification)
plt.show()

print(fortification.isnull().sum())

### Aerial Superiority

In [ ]:
aerial = df[['aeroa', 'wina']]
msno.matrix(aerial)
plt.show()
print(aerial.isnull().sum())

## Data Overview

### Wina Reference

    * -1 : attacker loss
    * 0 : draw
    * 1 : attaccker win

### Element of Surprise

### Element of Surprise reference

-3 ~ -1 : surprise achieved by defender(-3: most, -2: substantial , -1: minor)
    
0 : Neither Side surprised
    
1 ~ 3 : surprise achieved by attacker(3: most, 2: substantial , 1: minor)

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina',hue='surpa',data= surprise ,palette='RdBu_r')

### Terrain

### Terrain reference

terra 1:
    
    0 - None

    R - Rolling
    
    G - Rugged
    
    F - Flat
    
terra 2:
    
    0 - None
    
    B - Bare
    
    M - Mixed
    
    D - Desert
    
    W - Heavily Wooded
    
terra 3 :
    
    0 - None
    
    M - Marsh or Swamp
    
    U - Urban
    
    D - Dunes

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra1',data= terrain ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra2',data= terrain ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra3',data= terrain ,palette='RdBu_r')

### Weather

### Weather Reference

wx1:
    
    D - Dry
    W - Wet
    
wx2:
    
    H - Heavy Precipitatiion
    S - Sunny (no precipitation)
    L - Light Precipitation
    O - Overcast (no precipitation)
    
wx3:
    
    H - Hot
    C - Cold
    T - Temparate
    
wx4:

    S - Summer
    $ - Spring
    W - Winter
    F - Fall
    
wx5 : 
    
    E - Tropical (i.e., "Equatorial")
    D - Desert
    T - Temperate

In [ ]:
 for j in range(1, 6):
        
    plt.figure(figsize = (12, 12))
    sns.set_style('whitegrid')
    sns.countplot(x='wina', hue='wx{}'.format(j),data= weather ,palette='RdBu_r')


### Formation of Defense

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina', hue='post1',data= fortification ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina', hue='post2',data= fortification ,palette='RdBu_r')

### Aerial Superiority

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina',hue='aeroa',data= aerial ,palette='RdBu_r')

Based on my assesment of the datasets, I'll make the following changes to the tables.

    * If there are too many missing values for imputation, it will be excluded.(terra3, post2)
    
    
    * We'll set the train and test datasets together, using Kfold.
     
    * We'll drop the columns with na values except for 'terra1':0, because we cannot manipulate the historical facts. 
    We fill in 'wx2' missing values with the most frequent values coz there's too much missing values there.
    

In [ ]:
df_combined = df[['surpa', 'post1', 'wx1', 'wx2', 'wx3', 'wx4', 'wx5', 'terra1', 'terra2', 'aeroa', 'wina']]

display(df_combined)

msno.matrix(df_combined)
plt.show()
print(df_combined.isnull().sum())

In [ ]:
df_combined.columns

def asdf(*args):
    bomb = []
    for a in df_combined.columns:
        if a != 'wx2':
            bomb.append(a)
        
    return bomb

column_list = asdf(df_combined.columns)
print(column_list)

In [ ]:
df_combined = df_combined[column_list].dropna()
display(df_combined) #926 datas left
print(df_combined.isnull().sum())

In [ ]:
df_mask= df_combined['wina'] != 0
df_combined = df_combined[df_mask]
df_combined['wina']

 Coz of asymmetry problem of the dataset, drop all wina values with 0.(draw)
Our goal is to predict which side would win(attacker or defender), so dropping 'draw' values wouldn't be a problem.


## 1. Creating categories based on Defense Formation

 * DL - D - Delaying action adopted
 * WD - W - Withdrwal
 * FD - F - Fortified defense
 * HD - H - Hasty Defense
 * PD - P - Prepared Defense

In [ ]:
def check_post1():
    global df_combined
    
    df_combined['post1'] = df_combined['post1'].map(lambda ca: ca[0])
    #dummy encoding
    post1_dummies = pd.get_dummies(df_combined['post1'], prefix='post1')
    df_combined = pd.concat([df_combined, post1_dummies], axis=1)
    df_combined.drop('post1', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_post1()
display(df_combined)

print(df_combined['post1_D'].sum())
print(df_combined['post1_F'].sum())
print(df_combined['post1_H'].sum())
print(df_combined['post1_P'].sum())
print(df_combined['post1_W'].sum())

def check_FD():
    global df_combined
    df_combined['post1'] = df_combined['post1'].map(lambda s: 1 if s == 'FD' else 0)
    return df_combined

df_combined = check_FD()
display(df_combined)
print(df_combined['post1'].sum())

## 2. Creating categories based on wx1

 * 'D':Dry → 0
 * W':Wet → 1

In [ ]:
def check_wx1():
    global df_combined
    df_combined['wx1'] = df_combined['wx1'].map(lambda s: 1 if s == 'W' else 0)
    return df_combined

In [ ]:
df_combined = check_wx1()
display(df_combined)
print(df_combined['wx1'].sum())

## 3. Creating categories based on wx3

Every value is turned into a column including only 0 or 1 values.

* C : Cold
* H : Hot
* T : Temparate

In [ ]:
def check_wx3():
    global df_combined
    
    df_combined['wx3'] = df_combined['wx3'].map(lambda ca: ca[0])
    #dummy encoding
    wx3_dummies = pd.get_dummies(df_combined['wx3'], prefix='wx3')
    df_combined = pd.concat([df_combined, wx3_dummies], axis=1)
    df_combined.drop('wx3', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx3()
display(df_combined)
print(df_combined['wx3_C'].sum())
print(df_combined['wx3_H'].sum())
print(df_combined['wx3_T'].sum())

## 4. Creating categories based on wx4

Every value is turned into a column including only 0 or 1 values.

* S : Summer
* $ : Summer
* W : Winter
* F : Fall

In [ ]:
def check_wx4():
    global df_combined
    
    df_combined['wx4'] = df_combined['wx4'].map(lambda ca: ca[0])
    #dummy encoding
    wx4_dummies = pd.get_dummies(df_combined['wx4'], prefix='wx4')
    df_combined = pd.concat([df_combined, wx4_dummies], axis=1)
    df_combined.drop('wx4', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx4()
display(df_combined)
print(df_combined['wx4_S'].sum())
print(df_combined['wx4_$'].sum())
print(df_combined['wx4_W'].sum())
print(df_combined['wx4_F'].sum())

## 5. Creating categories based on wx5

Every value is turned into a column including only 0 or 1 values.

* E : Tropical (i.e., "Equatorial")
* D : Desert
* T : Temperate

In [ ]:
def check_wx5():
    global df_combined
    
    df_combined['wx5'] = df_combined['wx5'].map(lambda ca: ca[0])
    #dummy encoding
    wx5_dummies = pd.get_dummies(df_combined['wx5'], prefix='wx5')
    df_combined = pd.concat([df_combined, wx5_dummies], axis=1)
    df_combined.drop('wx5', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx5()
display(df_combined)
print(df_combined['wx5_E'].sum())
print(df_combined['wx5_D'].sum())
print(df_combined['wx5_T'].sum())

## 6. Creating categories based on terrain1

* R-Rolling
* G-Rugged
* F-Flat

In [ ]:
#creating function for creating categories based on the cabin of passengers
def check_terra1():
    global df_combined
    df_combined['terra1'] = df_combined['terra1'].map(lambda ca: ca[0])
    #dummy encoding
    terra1_dummies = pd.get_dummies(df_combined['terra1'], prefix='terra1')
    df_combined = pd.concat([df_combined, terra1_dummies], axis=1)
    df_combined.drop('terra1', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_terra1()
display(df_combined)
print(df_combined['terra1_R'].sum())
print(df_combined['terra1_G'].sum())
print(df_combined['terra1_F'].sum())

## 7. Creating categories based on terrain2

* B - Bare
* M - Mixed
* D - Desert
* W - Heavily wooded

In [ ]:
#creating function for creating categories based on the cabin of passengers
def check_terra2():
    global df_combined


    df_combined['terra2'] = df_combined['terra2'].map(lambda ca: ca[0])
    #dummy encoding
    terra2_dummies = pd.get_dummies(df_combined['terra2'], prefix='terra2')
    df_combined = pd.concat([df_combined, terra2_dummies], axis=1)
    df_combined.drop('terra2', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_terra2()
display(df_combined)
print(df_combined['terra2_B'].sum())
print(df_combined['terra2_M'].sum())
print(df_combined['terra2_D'].sum())
print(df_combined['terra2_W'].sum())

## 8. Creating categories based on aeiral superiority

* None, coz the values are already converted into integer

# **Building and Training the Models**

In [ ]:
df_combined['wina'] = df_combined['wina'].apply(lambda x: x+1 if x == -1 else x)
df_combined = df_combined.astype('int')
df_combined['wina'].unique()
# 0: attacker loss,  1: attacker win 

In [ ]:
from sklearn.model_selection import train_test_split

Train, Test = train_test_split(df_combined, test_size=0.2, random_state=123)

In [ ]:
!pip install pycaret
from pycaret.classification import *

## PyCaret installation**

In [ ]:
clf = setup(data = Train, target = 'wina')
# 'wina' is the target

## Model training and comparison

In [ ]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

## Model Ensemble

In [ ]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

## Prediction

In [ ]:
pred_holdout = predict_model(blended)

## Retraining on the dataset

In [ ]:
final_model = finalize_model(blended)

## Prediction on Test set


In [ ]:
Predictions = predict_model(final_model, data = Test)
Predictions

In [ ]:
Result = Predictions[['wina', 'Label']]
Result = Result.astype('int')
Result

In [ ]:
model_score = sum(Result['wina'] == Result['Label'])/len(Result)
model_score

# Summary

Though the model score was not high, I guess it attributes to the changing conditions of war, such as invention of aerial vehicle. Also the categorial differentiation with such climate, territorial conditions might not reflect the conditions perfectly. So my conclusion is that the dataset has some fundemental issues to deal with, and cannot be treated thoroughly with conventional machine learning.